In [379]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [380]:
class Activation:
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
        
    def sigmoid_derivative(self,x):
        return x * (1 - x)
    
    def relu(self,x):
        return np.maximum(0,x) 
    
    def relu_derivative(self,x):
        return 1. * (x > 0)

    def tanh(self,x):
        return np.tanh(x)

    def tanh_derivative(self,x):
        return 1 - x**2

    def softmax(self,x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum()

    def softmax_derivative(self,x):
        return self.softmax(x) * (1 - self.softmax(x))

    def cross_entropy(self,y_true, y_pred):
        return -np.sum(y_true * np.log(y_pred))

    def cross_entropy_derivative(self,y_true, y_pred):
        return -y_true/y_pred

    def mse(self,y_true, y_pred):
        return np.mean(np.square(y_true - y_pred))

    def mse_derivative(self,y_true, y_pred):
        return 2 * (y_pred - y_true) / y_true.size


In [381]:
class Conv2D:
    def __init__(self, n_filters, kernel_size, stride=1, padding=0, activation='relu'):
        self.n_filters = n_filters
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.activation = activation
        self.weights = np.random.randn(n_filters, kernel_size, kernel_size) / np.sqrt(kernel_size * kernel_size)
        self.bias = np.zeros((n_filters, 1))

    def forward(self, input):
        self.input = input
        n_filters, d_filter, h_filter, w_filter = self.weights.shape
        n_x, d_x, h_x, w_x = input.shape
        h_out = int((h_x - h_filter + 2 * self.padding) / self.stride + 1)
        w_out = int((w_x - w_filter + 2 * self.padding) / self.stride + 1)
        conv_out = np.zeros((n_x, n_filters, h_out, w_out))
        self.input_col = self.im2col_indices(input, h_filter, w_filter, padding=self.padding, stride=self.stride)
        self.weights_col = self.weights.reshape(n_filters, -1)
        out = self.weights_col @ self.input_col + self.bias
        out = out.reshape(n_filters, h_out, w_out, n_x)
        out = out.transpose(3, 0, 1, 2)
        conv_out = out
        self.conv_out = conv_out
        return self.activation(conv_out)
    


In [382]:
class Pool2D:
    def __init__(self, pool_size, stride=1, padding=0, mode='max'):
        self.pool_size = pool_size
        self.stride = stride
        self.padding = padding
        self.mode = mode

    def forward(self, input):
        self.input = input
        n_x, d_x, h_x, w_x = input.shape
        h_out = int((h_x - self.pool_size + 2 * self.padding) / self.stride + 1)
        w_out = int((w_x - self.pool_size + 2 * self.padding) / self.stride + 1)
        pool_out = np.zeros((n_x, d_x, h_out, w_out))
        for i in range(h_out):
            for j in range(w_out):
                h_start = i * self.stride
                h_end = h_start + self.pool_size
                w_start = j * self.stride
                w_end = w_start + self.pool_size
                x_slice = input[:, :, h_start:h_end, w_start:w_end]
                if self.mode == 'max':
                    pool_out[:, :, i, j] = np.max(x_slice, axis=(2, 3))
                elif self.mode == 'average':
                    pool_out[:, :, i, j] = np.mean(x_slice, axis=(2, 3))
        self.pool_out = pool_out

In [383]:
class Flatten:
    def __init__(self):
        self.input_shape = None

    def forward(self, input):
        self.input_shape = input.shape
        return input.reshape(input.shape[0], -1)

    def backward(self, d_output):
        return d_output.reshape(self.input_shape)

In [384]:
class Dense:
    def __init__(self, input_units, output_units, activation='relu'):
        self.weights = np.random.randn(input_units, output_units) / np.sqrt(input_units)
        self.bias = np.zeros((1, output_units))
        self.activation = activation
        self.activation_func = Activation()
        self.activation_func_name = activation
        self.activation_func_derivative_name = activation + '_derivative'
        self.activation_func_derivative = getattr(self.activation_func, self.activation_func_derivative_name)

    def forward(self, input):
        self.input = input
        self.output = np.dot(input, self.weights) + self.bias
        return self.output

    def backward(self, d_output):
        self.d_input = np.dot(d_output, self.weights.T)
        self.d_weights = np.dot(self.input.T, d_output)
        self.d_bias = np.sum(d_output, axis=0, keepdims=True)
        return self.d_input
    
    def update(self, learning_rate):
        self.weights -= learning_rate * self.d_weights
        self.bias -= learning_rate * self.d_bias
    
    def get_weights(self):
        return self.weights
    

In [385]:
# Create sequential model
class Sequential:
    def __init__(self, layers):
        self.layers = layers
        self.loss = None
        self.loss_prime = None
        self.input = None
        self.output = None
        self.y = None
        self.learning_rate = None

    def forward(self, input):
        self.input = input
        self.output = input
        for layer in self.layers:
            self.output = layer.forward(self.output)
        return self.output

    def backward(self, d_output):
        for layer in reversed(self.layers):
            d_output = layer.backward(d_output)
        return d_output

    def set(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

    def train(self, x_train, y_train, x_test, y_test, epochs, learning_rate, batch_size=int):
        self.set(learning_rate=learning_rate)
        train_loss = []
        test_loss = []
        for i in range(epochs):
            print('Epoch {}/{}'.format(i + 1, epochs))
            print('-' * 10)
            for j in range(0, len(x_train), batch_size):
                x_batch = x_train[j:j + batch_size]
                y_batch = y_train[j:j + batch_size]
                self.train_on_batch(x_batch, y_batch)
            train_loss.append(self.evaluate(x_train, y_train))
            test_loss.append(self.evaluate(x_test, y_test))
            print('train loss: {}, test loss: {}'.format(train_loss[-1], test_loss[-1]))
        return train_loss, test_loss
        # for epoch in range(epochs):
        #     output = self.forward(x_train)
        #     loss = self.loss(y_train, output)
        #     accuracy = self.accuracy(y_train, output)
        #     d_output = self.loss_prime(y_train, output)
        #     self.backward(d_output)
        #     self.update()
        #     if epoch % 10 == 0:
        #         print('Epoch: {}, Loss: {:.3f}, Accuracy: {:.3f}'.format(epoch, loss, accuracy))
        #         print('Test loss: {:.3f}, Test accuracy: {:.3f}'.format(self.test(x_test, y_test), self.accuracy(y_test, self.forward(x_test))))

    def update(self):
        for layer in self.layers:
            if hasattr(layer, 'weights'):
                layer.weights -= self.learning_rate * layer.d_weights
            if hasattr(layer, 'bias'):
                layer.bias -= self.learning_rate * layer.d_bias

    def test(self, x_test, y_test):
        output = self.forward(x_test)
        loss = self.loss(y_test, output)
        return loss

    def accuracy(self, y_true, y_pred):
        y_pred = np.argmax(y_pred, axis=1)
        y_true = np.argmax(y_true, axis=1)
        accuracy = np.mean(y_pred == y_true)
        return accuracy

In [386]:
import glob
from sklearn.model_selection import train_test_split

# Read images and convert them to arrays
image_size = (100,100)

def read_image_array(path):
    path = glob.glob(path)
    images = []

    for image in path:
        img = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, image_size)
        images.append(img)
    return images

cats = read_image_array('./cats/*.jpg')
dogs = read_image_array('./dogs/*.jpg')

# Make an array of 0s for cats and 1s for dogs
X = np.array(cats + dogs)
y = np.array([0] * len(cats) + [1] * len(dogs))

# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1000)

# Normalize the data
X_train = X_train / 255
X_test = X_test / 255

# Reshape the data
X_train = X_train.reshape(-1, 100, 100, 1)
X_test = X_test.reshape(-1, 100, 100, 1)

# Create the model
model = Sequential([
    Conv2D(32, 3, stride = 1, activation=Activation.relu),
    Pool2D(2, 2),
    Conv2D(64, 3, stride = 1, activation=Activation.relu),
    Pool2D(2, 2),
    Flatten()
    ])

model.train(X_train, y_train, X_test, y_test, 2, 0.01, 0.9)

Epoch 1/2
----------


TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
# Kelas Dense digunakan untuk membuat model yang lebih kompleks. Kita dapat menambahkan layer Dense setelah Flatten layer untuk membuat model yang lebih kompleks.
# Kelas Activation digunakan untuk menambahkan fungsi aktivasi pada model. Kita dapat menambahkan fungsi aktivasi pada layer Conv2D, MaxPooling2D, dan Dense layer.
# Kelas Dropout digunakan untuk mengurangi overfitting pada model. Kita dapat menambahkan layer Dropout setelah layer Conv2D, MaxPooling2D, dan Dense layer.
# Kelas Flatten digunakan untuk mengubah dimensi dari hasil pooling menjadi 1 dimensi. Kita dapat menambahkan layer Flatten setelah layer MaxPooling2D.
# Kelas Conv2D digunakan untuk membuat layer konvolusi pada model. Kita dapat menambahkan layer Conv2D setelah layer Input.
# Kelas MaxPooling2D digunakan untuk membuat layer pooling pada model. Kita dapat menambahkan layer MaxPooling2D setelah layer Conv2D.